In [1]:
#import basic library
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
import re

In [2]:
#import library for mutiple threading
import _thread
import threading

In [3]:
#filechecker function
def filechecker(name):
    data=pd.read_csv(name)
    print('top 5 of',len(data),'rows from',name, 'data\n',data.head())

In [4]:
#swg_crawler function for creating brief info for switch games

#global item_link
item_link=[]
def swg_crawler(link,filename='swg_brief.csv'):
    response=requests.get(link)
    soup=BeautifulSoup(response.text,'html.parser')
    
    games=soup.find_all('div',{'class':'cell'})
    games
    for game in games:
        describe=game.find('div',{'class':'h6 name'}).text.strip()

        try:
            describe.split('\n')[0]
            name=describe.split('\n')[0]
        except:
            name=[]
            
        try:
            describe.split('\n')[1]
            discount=describe.split('\n')[1]
        except:
            discount=[]
            
        try:
            describe.split('\n')[2]
            tag=describe.split('\n')[2]
        except:
            tag=[]
            
        try:
            original_price=game.find('s',{'class':'text-muted'}).text.strip()
        except:
            original_price=[]
            
        try:
            discount_price=game.find('strong').text.strip()
        except:
            discount_price=[]
            
        try:
            deadline=game.find('div',{'class':'w-100'}).find('small').text.strip()
        except:
            deadline=[]
            
        
        link=game.find('a',{'class':'main-link'})['href']
        global item_link
        item_link.append('https://www.dekudeals.com'+link)
        
        with open(filename,"a+",encoding='utf-8') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow([name,discount,tag,original_price,discount_price,deadline])
        

In [5]:
#swg_crawler function for creating detail info for switch games

def swg_item_crawler(link,filename='swg_detail.csv'):
    response=requests.get(link)
    soup=BeautifulSoup(response.text,'html.parser')
    name=soup.find('span',{'class':'display-5'}).text
    info=soup.find_all('li',{'class':'list-group-item'})
    genre=[]
    publisher=[]
    size=[]
    release_date=[]
    metascore=[]
    userscore=[]
    language=[]
    play_mode=[]
    
    for i in info:
        if 'Genre' in i.text:
            genre= i.text.split(': ')[1:]
            #print(genre)
        if 'Publisher' in i.text:
            publisher=i.text.split(': ')[1:]
            #print(publisher)
        if 'Download' in i.text:
            size=i.text.split(': ')[1:]
            #print(size)
        if 'Release' in i.text:
            release_date=i.text.split(': ')[1:]
            #print(release_date)
        if 'Metacritic' in i.text:
            metascore=i.text.split(' ')[1]
            userscore=i.text.split(' ')[2]
            #print(metascore)
            #print(userscore)
        if 'Language' in i.text:
            language=i.text.split(': ')[1:]
            #print(language)
        if 'Play modes' in i.text:
            play_mode=i.text.split(': ')[1:]
            #print(play_mode)
    stores=soup.find_all('tr',{'class':'table-primary'})
    loc={}
    for store in stores:
        try:
            store_loc=store.find('img')['alt']
            version=store.find('td',{'class':'version'}).text.strip()
            price,discount=(store.find('div',{'class','btn-primary'}).text.strip().split('\n'))
            loc.update({store_loc:[version,price,discount]})
        except:
            pass
    #print(loc)
    with open(filename,"a+",encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow([name,genre,publisher,size,release_date,metascore,userscore,language,play_mode,loc])

In [6]:
#create header for files
with open('swg_brief.csv',"a+",encoding='utf-8') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow(['Name','Discount','Tag','Original_Price','Discount_Price','Deadline'])

with open('swg_detail.csv',"a+",encoding='utf-8') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow(['Name','Genre','Tag','Publisher','Download_Size','Release_Date','MetaScroe','UserScroe','Language','Play_Modes'])

In [7]:
#get the largest page number
link='https://www.dekudeals.com/hottest?page=1'
r=requests.get(link)
soup=BeautifulSoup(r.text,'html5lib')
maxpage=int(soup.find_all('a',{'class':'page-link'})[-2].text)+1
maxpage

17

In [8]:
main='https://www.dekudeals.com/'
link_list=[]
for i in range(1,maxpage):
    link_list.append(main+'hottest?page='+str(i))
link_list

['https://www.dekudeals.com/hottest?page=1',
 'https://www.dekudeals.com/hottest?page=2',
 'https://www.dekudeals.com/hottest?page=3',
 'https://www.dekudeals.com/hottest?page=4',
 'https://www.dekudeals.com/hottest?page=5',
 'https://www.dekudeals.com/hottest?page=6',
 'https://www.dekudeals.com/hottest?page=7',
 'https://www.dekudeals.com/hottest?page=8',
 'https://www.dekudeals.com/hottest?page=9',
 'https://www.dekudeals.com/hottest?page=10',
 'https://www.dekudeals.com/hottest?page=11',
 'https://www.dekudeals.com/hottest?page=12',
 'https://www.dekudeals.com/hottest?page=13',
 'https://www.dekudeals.com/hottest?page=14',
 'https://www.dekudeals.com/hottest?page=15',
 'https://www.dekudeals.com/hottest?page=16']

In [9]:
threads=[]
#build all the tasks 
for link in link_list:
    thread = threading.Thread(target=swg_crawler,args=(link,'swg_brief.csv'))
    threads.append(thread) 
 
                              
startTime=time.time()                              
#start all the tasks at once
for thread in threads:
    thread.start()
    
#wait for all the tasks to finish
for thread in threads:
    thread.join()
                              
finishTime = time.time()
print(finishTime - startTime,'secs spend to craw these {0} pages'.format(len(link_list)))

      
filechecker('swg_brief.csv')


8.061360836029053 secs spend to craw these 16 pages
top 5 of 558 rows from swg_brief.csv data
                                         Name Discount                   Tag  \
0                  Tower of Babel - no mercy     -80%     Lowest price ever   
1  LEGO Marvel Super Heroes 2 Deluxe Edition     -70%  Matches previous low   
2                                 1-2-Switch     -28%                    []   
3                        Stick It to The Man     -80%  Matches previous low   
4                      Wide Ocean Big Jacket     -60%  Matches previous low   

  Original_Price Discount_Price              Deadline  
0          $9.99          $1.99  Sale ends in 7 hours  
1         $44.99         $13.49  Sale ends November 2  
2         $49.99         $35.88                    []  
3         $11.99          $2.39  Sale ends November 1  
4          $7.99          $3.19  Sale ends November 1  
